# Steps

In [1]:
# Importing libraries
# Reading dataset
# understand dataset
# update dataset
# get canonical dealerships
# Join original and updated datasets to check updated values
# Get quality % as per NA count for each tenant
# collect all issues and corrections
# model creation
# model serialization for deployment
# QnA

# Importing libraries

In [2]:
import numpy as np
import pandas as pd
import re
import os


In [3]:
# function to get dataframe report
def report(data):
    data_types = pd.DataFrame(
        data.dtypes,
        columns=['Data Type']
    )
    missing_data = pd.DataFrame(
        data.isnull().sum(),
        columns=['Missing Values']
    )
    unique_values=pd.DataFrame(
        data.nunique(),
        columns=['Unique Values']
    )
    base=data_types.join(missing_data).join(unique_values)
    base['Unique values %']=base[['Unique Values']].applymap(lambda x:x*100/data.shape[0])
    base['Missing values %']=base[['Missing Values']].applymap(lambda x:x*100/data.shape[0])
    return base

In [4]:
# reading dataset
dataset_org=pd.read_csv('dataset.csv')
# creating dataset copy for operations
dataset=dataset_org.copy()

# Understanding Dataset

In [5]:
# printing report off original/old dataset
report_dataset_org=report(dataset_org)
report_dataset_org

,Data Type,Missing Values,Unique Values,Unique values %,Missing values %
id,int64,0,29668,98.893333,0.000000
name,object,0,9158,30.526667,0.000000
tenant_id,int64,0,661,2.203333,0.000000
address_line,object,2,8538,28.460000,0.006667
city,object,0,2522,8.406667,0.000000
state,object,0,53,0.176667,0.000000
zip,object,1,4267,14.223333,0.003333
country,object,0,1,0.003333,0.000000
phone,object,1883,7658,25.526667,6.276667
web,object,12642,8786,29.286667,42.140000


In [6]:
# checking dataset shape
dataset.shape

(30000, 13)

In [7]:
# checking dataset format
dataset.head()

,id,name,tenant_id,address_line,city,state,zip,country,phone,web,google_url,facebook_url,cars_url
0,3244033,# 1 Brookville Chevrolet,130469,1 E Main St,Brookville,PA,15825,US,(814) 849-8313,https://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=1262205935310...,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookvil...
1,18311,#1 BROOKVILLE CHEVROLET-BUICK,1840,ONE E MAIN ST,BROOKVILLE,PA,15825,US,(814) 849-8313,NaN,NaN,https://www.facebook.com/brookvillechevy/,NaN
2,118193,#1 BROOKVILLE CHEVROLET-BUICK,2980,1 East Main St,Brookville,PA,15825,US,(814) 849-8313,NaN,https://maps.google.com/maps?cid=1262205935310...,https://www.facebook.com/brookvillechevy/,http://www.cars.com/dealers/154863/1-brookvill...
3,18248,#1 BROOKVILLE CHEVROLET-BUICK,1836,ONE E MAIN ST,BROOKVILLE,PA,15825,US,(814) 849-8313,NaN,NaN,NaN,NaN
4,575489,#1 BROOKVILLE CHEVROLET-BUICK,206,30 Main St,Brookville,PA,15825,US,(814) 849-8313,http://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=1262205935310...,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookvil...


In [8]:
# count na values in original dataset
na_values_total_dataset_org=report_dataset_org['Missing Values'].sum()

# Updating dataset

In [9]:
# converting values to uppercase to get proper calculation in unique values
dataset['city']=dataset['city'].str.upper()
dataset['address_line']=dataset['address_line'].str.upper()
dataset['name']=dataset['name'].str.upper()

In [10]:
# filling all NaN values with string NA to get proper unique values
dataset.fillna('NA',inplace=True)

# Get Canonical dataset

In [11]:
# generate dataset with canonical rows by removing duplicate ids
dataset_canonical=dataset.drop_duplicates(subset='id',inplace=False,keep='first')

In [12]:
# checking dataset_canonical shape
dataset_canonical.shape

(29668, 13)

In [13]:
no_of_duplicates=dataset_org.shape[0]-dataset_canonical.shape[0]
# (no_of_duplicates)

# Join old and new datasets to check updated values

In [14]:
# old dataset columns with suffix _current 
# new dataset columns with suffix _new 

dataset_concat=pd.concat([dataset_org.add_suffix('_current'),dataset.add_suffix('_new')],axis=1)


In [15]:
# keeping id columns at left
# even if suffixes are given, respective values are same in both cols
cols=sorted(dataset_concat.columns)#
cols.remove('id_current')#.
cols.remove('id_new')
cols_sorted=['id_current','id_new']+cols
dataset_concat_sorted_cols=dataset_concat.reindex(cols_sorted, axis=1)


In [16]:
# writing joined dataset to csv
dataset_concat_sorted_cols.to_csv('dataset_concat.csv')

# Getting corrections

In [17]:
# get report of canonical dataset
report_dataset_new=report(dataset_canonical)
report_dataset_new

,Data Type,Missing Values,Unique Values,Unique values %,Missing values %
id,int64,0,29668,100.000000,0.0
name,object,0,8189,27.602130,0.0
tenant_id,int64,0,661,2.227990,0.0
address_line,object,0,8066,27.187542,0.0
city,object,0,1895,6.387353,0.0
state,object,0,53,0.178644,0.0
zip,object,0,4268,14.385870,0.0
country,object,0,1,0.003371,0.0
phone,object,0,7659,25.815694,0.0
web,object,0,8787,29.617770,0.0


In [18]:
# calculate total no of corrected unique values in name, city, address_line column
values_updated=dict(report_dataset_org['Unique Values']-report_dataset_new['Unique Values'])
unique_values_updated_by_uppercase=dict()
for i in values_updated.keys():
    if i in ('name','city','address_line'):
        unique_values_updated_by_uppercase[i]=values_updated[i]

# Get CSV of Data quality by tenant

In [19]:
# creating dataframe groups by tenant_id
tenant_id_group=dataset_canonical.groupby(['tenant_id'])

In [20]:
# checking no of groups
len(tenant_id_group)

661

In [21]:
# create dataframe named 'quality' to check quality as % of NA values count for each group
quality=pd.DataFrame(columns=['tenant id','na_count','total count'])
for key, df in tenant_id_group:
    na_count=0
    total_count=df.shape[1]*df.shape[0]
    for col in df.columns:
        if col not in ('id','tenant_id'):
            try:
                na_count=df[col].value_counts()['NA']+na_count
            except:
                pass
    quality=quality.append(pd.Series({'tenant id':key,'na_count':na_count,'total count':total_count}),ignore_index=True)    # na_dict[key]=na_count


In [22]:
# create quality column as % of count of NA in each group
# lesser the NA count, Higher the quality of data
quality['quality%_by_NA_count']=100-(quality['na_count']*100/quality['total count'])

In [23]:
# write quality to csv
quality.to_csv('quality.csv')

# Generating report

In [24]:
# Total number of canonical locations
unique_addresses_group=dataset_canonical.groupby(['address_line','city','zip','state','country'])
total_no_of_canonical_locations=len(unique_addresses_group)

In [25]:
# writing report.txt
with open('report.txt','w')as f:
    f.write(f'Total number of distinct dealerships (locations ids): {len(dataset_canonical)}\n')
    f.write(f'Total number of canonical locations: {total_no_of_canonical_locations}\n')
    f.write(f'Total number of data issues (missing cells) found  : {na_values_total_dataset_org}\n')
    f.write(f'Total number of data corrections:\n')
    f.write(f'total no of duplicate rows removed: {no_of_duplicates}\n')
    f.write(f'total no of NA values filled: {na_values_total_dataset_org}\n')
    f.write(f'total no of unique values corrected by changing case in following columns: {unique_values_updated_by_uppercase}\n')
    

# Model creation

In [ ]:
# # creating model to predict the closest possible dealership from canonical dataset

# encoding from string to num

# from sklearn.preprocessing import OrdinalEncoder
# oe=OrdinalEncoder()
# dataset_train=dataset_canonical.copy()

# from sklearn.compose import ColumnTransformer

# # encoding only those variables which essential for dealership and distinct location

# transformer = ColumnTransformer(transformers=[
#     ('tnf1',OrdinalEncoder(),['name']),
#     ('tnf2',OrdinalEncoder(),['tenant_id']),
#     ('tnf3',OrdinalEncoder(),['address_line']),
#     ('tnf4',OrdinalEncoder(),['city']),
#     ('tnf5',OrdinalEncoder(),['state']),
#     ('tnf6',OrdinalEncoder(),['zip'])

# ],remainder='drop')

# dataset_train_trf=transformer.fit_transform(dataset_train)


In [ ]:
# # creating clusters of number=distinct dealerhsips
# # this will predict the future incoming deals closest to one from canonical dataset

# from sklearn.cluster import KMeans
# kmeans=KMeans(len(dataset_canonical))
# kmeans.fit(dataset_train_trf)

In [ ]:
# # predict closest possible dealership

# kmeans.predict([future_dealership])

# Model/object serilization

In [26]:
# import joblib
# joblib.dump('model.pkl',model)

# QnA

In [27]:
"""
Explain any 2 methods that you would prefer to push the project to production 
for client usage

Ans:
Method 1:
step 1: Creating repository and pushing it to github with following files

app.py: flask application
model.pkl: model
train.py: training script
test.py: testing script
dataset.csv: training data
README.md: README
config.yaml: server configuration varaibles
deployment.yaml: deployment configuration variables, docker build,push,run
Dockerfile: docker config file
requirements.txt: python libraries to install
index.html: HTML homepage of web application
procfile: server config file

step 2: Connect github repo to GCP source repo with push trigger
step 3: Set external IP visible to all
step 4: create kubernetes cluster and configure with current project

Method 2:

"""

'\nExplain any 2 methods that you would prefer to push the project to production \nfor client usage\n\nAns:\nMethod 1:\nstep 1: Creating repository and pushing it to github with following files\n\napp.py: flask application\nmodel.pkl: model\ntrain.py: training script\ntest.py: testing script\ndataset.csv: training data\nREADME.md: README\nconfig.yaml: server configuration varaibles\ndeployment.yaml: deployment configuration variables, docker build,push,run\nDockerfile: docker config file\nrequirements.txt: python libraries to install\nindex.html: HTML homepage of web application\nprocfile: server config file\n\nstep 2: Connect github repo to GCP source repo with push trigger\nstep 3: Set external IP visible to all\nstep 4: create kubernetes cluster and configure with current project\n\nMethod 2:\n\n'